<a href="https://colab.research.google.com/github/Jaskeerat23/DSA_codes/blob/main/X_Sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git remote add origin "https://github.com/Jaskeerat23/Sentiment_Analysis.git"

In [2]:
# !git status

In [30]:
!kaggle competitions download -c text-normalization-challenge-english-language

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


# **Importing Libraries**

In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
import math

from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

# **Mounting Drive**

This is because the glove embeddings are installed on my drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# **Reading Dataset**

In [5]:
df = pd.read_csv('/content/twitter_training.csv', header = None)

df.columns = ['id', 'topic', 'sentiment', 'tweet']

df.head()

,id,topic,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


# **Preprocessing**

# **Converting to String**

In [6]:
def to_string(sentence):
  return str(sentence)
df['tweet'] = df['tweet'].apply(to_string)

# **Lowering the alphabets**

In [7]:
df['tweet'] = df['tweet'].apply(str.lower)
df.head()

,id,topic,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


# **Removing HTML tags and urls**

In [8]:
def remove_tags(sentence):
  sentence = re.sub(r'<.*?>', '', sentence)
  return sentence

def remove_urls(sentence):
  sentence = re.sub(r'https://\S+|www\.\S+', '', sentence)
  return sentence

eg1 = "<html> actor <html> <br>"
eg2 = '''<html>
      <br>
      ok
      <br\>
      <html\>'''

eg3 = "www.html.com i am good"

print(f"eg1 : {remove_tags(eg1)}")
print(f"eg2 : {remove_tags(eg2)}")
print(f"eg3 : {remove_urls(eg3)}")

eg1 :  actor  
eg2 : 
      
      ok
      
      
eg3 :  i am good


In [9]:
df['tweet'] = df['tweet'].apply(remove_tags)
df['tweet'] = df['tweet'].apply(remove_urls)
df.head()

,id,topic,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


# **Filtering in only ALPHABETS**

In [10]:
def filter_alphabets(sentence : str):
  sentence = re.sub(r'[^a-zA-Z ]', ' ', sentence)
  return sentence

eg1 = "My name is jaskeerat singh and age is 20"
eg2 = "Today's date is 15/5/2025"

print(filter_alphabets(eg1))
print(filter_alphabets(eg2))

My name is jaskeerat singh and age is   
Today s date is          


In [11]:
df['tweet'] = df['tweet'].apply(filter_alphabets)
df['tweet'].head()

,tweet
0,im getting on borderlands and i will murder yo...
1,i am coming to the borders and i will kill you...
2,im getting on borderlands and i will kill you ...
3,im coming on borderlands and i will murder you...
4,im getting on borderlands and i will murder ...


# **Removing Stopwords**

In [12]:
def remove_stopwords(sentence : str):
  stopwords = set(nltk.corpus.stopwords.words('english'))
  sentence = re.sub(r'\.', ' ', sentence)
  sentence = sentence.split()
  sent = ""
  for word in sentence:
    if word not in stopwords:
      sent+=word + " "
  return sent[:len(sent)-1]
remove_stopwords('My name is Jaskeerat Singh . I am 19 yrs old')

'My name Jaskeerat Singh I 19 yrs old'

In [13]:
df['tweet'] = df['tweet'].apply(remove_stopwords)
df.head()

,id,topic,sentiment,tweet
0,2401,Borderlands,Positive,im getting borderlands murder
1,2401,Borderlands,Positive,coming borders kill
2,2401,Borderlands,Positive,im getting borderlands kill
3,2401,Borderlands,Positive,im coming borderlands murder
4,2401,Borderlands,Positive,im getting borderlands murder


# **Removing continous Duplicate words**

In [14]:
def remove_cont_dup(sentence):
  prev_word = ""
  new_sentence = []
  for word in sentence.split():
    if prev_word != word:
      new_sentence.append(word)
    prev_word  = word

  return " ".join(new_sentence)

eg1 = "really overwatch rn game sea sea sea sea sea sea sea sea sea sea sea sea sea sea sea sea sea sea sea see e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e"
eg2 = "Jaskeerat singh"

print(remove_cont_dup(eg1))
print(remove_cont_dup(eg2))

really overwatch rn game sea see e
Jaskeerat singh


In [15]:
df['tweet'] = df['tweet'].apply(remove_cont_dup)
df['tweet'].head()

,tweet
0,im getting borderlands murder
1,coming borders kill
2,im getting borderlands kill
3,im coming borderlands murder
4,im getting borderlands murder


# **Removing non-English or non-sense words**

In [31]:
def remove_noneng_words(sentence):
  sentence = sentence.lower()
  words = set([str.lower(w) for w in nltk.corpus.words.words()])
  new_sentence = []
  for word in sentence.split():
    if word in words:
      new_sentence.append(word)

  return " ".join(new_sentence)

eg1 = "MY name is jaskeerat singh eofnfWEGWEg"
eg2 = "smjhkl is anmfkwe good goood"

print(remove_noneng_words(eg1))
print(remove_noneng_words(eg2))

my name is singh
is good


In [ ]:
df['tweet'] = df['tweet'].apply(remove_noneng_words)
df['tweet']

# **Removing Single words except 'I' or 'i'(to be changed)**

In [16]:
def remove_single_words(sentence):
  new_sentence = []
  for word in sentence.split():
    if len(word) == 1 and (word == 'I' or word == 'i'):
      new_sentence.append(word)
    elif len(word) > 1:
      new_sentence.append(word)
  return " ".join(new_sentence)

eg1 = "Hey gpt e e e e e e"
eg2 = "i am jas"

print(remove_single_words(eg1))
print(remove_single_words(eg2))

Hey gpt
i am jas


In [17]:
df['tweet'] = df['tweet'].apply(remove_single_words)
df['tweet'].head()

,tweet
0,im getting borderlands murder
1,coming borders kill
2,im getting borderlands kill
3,im coming borderlands murder
4,im getting borderlands murder


# **Extracting text**

In [18]:
def extract_text(df):
  strings = []
  for string in df['tweet']:
    string = string.split()
    string = string[:25]
    string = " ".join(string)
    strings.append(string)
  return strings

tweets = extract_text(df)

# **Getting Max Length**

In [19]:
def max_length(tweets):
  max_len = -math.inf
  idx = 0

  for i, string in enumerate(tweets):
    if max_len < len(string.split()):
      max_len = len(string.split())
      idx = i

    max_len = max(len(string.split()), max_len)
  return max_len, idx

In [20]:
max_len, i = max_length(tweets)
print(f"Sentence max_len is {max_len}")
print(f"Sentence {i} is {tweets[i]}")

Sentence max_len is 25
Sentence 6 is spent hours making something fun know huge borderlands fan maya one favorite characters decided make wallpaper pc original image versus creation made enjoy pic twitter


# **Making set of words**

In [21]:
def make_set(tweets):
  words = set()
  for tweet in tweets:
    for word in tweet.split():
      words.add(word)
  return words

words = make_set(tweets)
len(words)

28676

In [22]:
def word_to_idx(words : set):
  word_idxs = {'<UNK>' : 0, '<PAD>' : 1}
  for idx, word in enumerate(words, start = 2):
    word_idxs.update({word : idx})

  return word_idxs

word_idxs = word_to_idx(words)
word_idxs

{'<UNK>': 0,
 '<PAD>': 1,
 'surfa': 2,
 'considering': 3,
 'creates': 4,
 'letter': 5,
 'deshaun': 6,
 'mofarahdota': 7,
 'funnymatters': 8,
 'cruise': 9,
 'theorists': 10,
 'vpesports': 11,
 'mowed': 12,
 'theyre': 13,
 'platforms': 14,
 'cyberbullying': 15,
 'navy': 16,
 'sports': 17,
 'wagner': 18,
 'provlib': 19,
 'conspiring': 20,
 'shootout': 21,
 'invzwqgrxw': 22,
 'dallas': 23,
 'rdo': 24,
 'silverhand': 25,
 'rumored': 26,
 'wsv': 27,
 'ybwrvfg': 28,
 'abusing': 29,
 'unfinished': 30,
 'maniacsega': 31,
 'censoredcinder': 32,
 'sflcqqqjmh': 33,
 'throes': 34,
 'perioperative': 35,
 'discrete': 36,
 'scpes': 37,
 'degrees': 38,
 'denounce': 39,
 'seok': 40,
 'devoted': 41,
 'nationalism': 42,
 'loser': 43,
 'revelling': 44,
 'mmbi': 45,
 'russell': 46,
 'vince': 47,
 'unsafe': 48,
 'exhibit': 49,
 'sharah': 50,
 'haircut': 51,
 'teodosianbard': 52,
 'cqc': 53,
 'caleb': 54,
 'jondaemal': 55,
 'zquez': 56,
 'hooks': 57,
 'petro': 58,
 'boyz': 59,
 'richardturrin': 60,
 'uncatego

In [23]:
if 'example' in nltk.corpus.words.words():
  print("yes")
else:
  print("no")

yes
